In [2]:
# Install spark-related dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d29d6332d5d9ed1d146f71031aec440fd81e1974ad9a2ee0bad77077be6bc3f7
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
# Point Colaboratory to your Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Download datasets directly to your Google Drive "Colab Datasets" folder запускается один раз данный код
import requests

# 2007 data

file_url = "https://gist.github.com/netj/8836201#file-iris-csv"

r = requests.get(file_url, stream = True) 

with open("/content/gdrive/My Drive/Colab Datasets/iris.csv.bz2", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block)

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark-ml').getOrCreate()

In [20]:
iris_df = spark.read.csv("/content/gdrive/My Drive/Colab Datasets/Iris.csv", header=True, inferSchema=True)
iris_df.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



In [21]:
iris_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [22]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [23]:
vectorAssembler = VectorAssembler(inputCols = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'], outputCol = 'features')
v_iris_df = vectorAssembler.transform(iris_df)
v_iris_df.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|         features|
+---+-------------+------------+-------------+------------+-----------+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+---+-------------+------------+-------------+------------+-----------+-----------------+
only showing top 5 rows



In [24]:
indexer = StringIndexer(inputCol = 'Species', outputCol = 'label')
i_v_iris_df = indexer.fit(v_iris_df).transform(v_iris_df)
i_v_iris_df.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------------+-----+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|         features|label|
+---+-------------+------------+-------------+------------+-----------+-----------------+-----+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|  0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|  0.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|  0.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|  0.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|  0.0|
+---+-------------+------------+-------------+------------+-----------+-----------------+-----+
only showing top 5 rows



In [25]:
i_v_iris_df.select('Species','label').groupBy('Species','label').count().show()

+---------------+-----+-----+
|        Species|label|count|
+---------------+-----+-----+
|    Iris-setosa|  0.0|   50|
| Iris-virginica|  2.0|   50|
|Iris-versicolor|  1.0|   50|
+---------------+-----+-----+



In [27]:
splits = i_v_iris_df.randomSplit([0.6,0.4],1)
train_df = splits[0]
test_df = splits[1]
train_df.count(), test_df.count(), i_v_iris_df.count()

(98, 52, 150)

In [28]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
layers = [4,5,5,3]
mlp = MultilayerPerceptronClassifier(layers = layers, seed = 1)

In [30]:
mlp_model = mlp.fit(train_df)

In [31]:
pred_df = mlp_model.transform(test_df)
pred_df.select('Id','features','label','rawPrediction','probability','prediction').show(5)

+---+-----------------+-----+--------------------+--------------------+----------+
| Id|         features|label|       rawPrediction|         probability|prediction|
+---+-----------------+-----+--------------------+--------------------+----------+
|  1|[5.1,3.5,1.4,0.2]|  0.0|[35.6966653222553...|[0.99999999998843...|       0.0|
|  5|[5.0,3.6,1.4,0.2]|  0.0|[35.6966656154644...|[0.99999999998843...|       0.0|
|  6|[5.4,3.9,1.7,0.4]|  0.0|[35.6966657795881...|[0.99999999998843...|       0.0|
| 10|[4.9,3.1,1.5,0.1]|  0.0|[35.6966639036960...|[0.99999999998843...|       0.0|
| 14|[4.3,3.0,1.1,0.1]|  0.0|[35.6966707833717...|[0.99999999998843...|       0.0|
+---+-----------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [32]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'accuracy')
mlpacc = evaluator.evaluate(pred_df)
mlpacc

0.9615384615384616